Run, develop and demonstrate a pipeline for searching for potential lensed supernovae.

Steps:
* Download public ZTF alerts distributed any particular day.
* Filter these based on the minimal criteria of potential detections.
* Obtain redshift information from number of sources: masterlens, NED, SDSS photo-z, LS photo-z
* Fit SALT2 model on best redshift estimate, return estimated absolute peak magnitude
* Plot lightcurve fit + links to slack

Note that a full alert archive, with a size of up to 20 GB will be downloaded. Files are retrieved from:
https://ztf.uw.edu/alerts/public/
and will be put in the notebook directory. For trial runs, choose a day with less alerts (e.g. 20210516)

The code assumes that the Ampel libraries are installed. Three steps are needed for this:
* Start a local MongoDB instance (with default connections).
* Download AMPEL v 0.8 core, ZTF and ampel-hu-astro libraries.
* Create a default ampel configuration file through running 'ampel-config build > ampel_conf.yaml'.

In [ ]:
import sys, os

In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext


In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel82/ampel_conf_60a371.yaml'

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
                              'mongo/logger':{},
                              'mongo/writer':{},
                              'extcats/reader':{},
            'ztf/archive/auth' : {'user' : '', 'password' : ''},
            'datalab/pwd':os.environ["DATALABPWD"],
            "desycloud" : {},
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
# Select date of query with format e.g. '20210610'
channel = 'GeneralLens'
ITER_MAX = 1000000
ALERT_ARCHIVE = '/home/jnordin/data/ztfalerts/ztf_public_20210926.tar.gz'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'Lens82',
    purge_db = False,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
# These are configuration parameters for the initial alert filter step. Changing these will accept more/less
# alerts.
filter_config = {
    'min_ndet': 6,
    'min_tspan': 5,
    'max_tspan': 50,
    'min_archive_tspan': 5,
    'max_archive_tspan': 50,    
    'min_rb': 0.3,
    'min_drb': 0.995,
    'max_fwhm': 5.5,
    'min_gal_lat': 14,
    'ps1_sgveto_rad': 1,
    'ps1_sgveto_th': 0.8,
    'ps1_confusion_rad': 3,
    'ps1_confusion_sg_tol': 0.1
}

In [ ]:
# Will use NED for spectroscopic redshifts.
cat_conf = {
    'catalogs' : {
        'SDSS_spec' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['z', 'bptclass', 'subclass'],
            'all': True,
        },
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'GLADEv23' : {
            'use' : 'extcats',
            'rs_arcsec' : 10,
            'keys_to_append' : ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': True,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },

    }
}

In [ ]:
# We with to digest available redshift to one used for lightcurve template
# the "Ampel" z weighs available matches and returns the mean of the most accurate ones (if available)
ampelz_conf = {
    "max_redshift_category" : 6,
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": cat_conf,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },
    ]
}

In [ ]:
# Finally, we perform a salt-fit using the aggregated data and the redshift from above
sncosmo_conf = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
    "plot_props": {
        "tags": ["SALT", "SNCOSMO"],
        "file_name": {
            "format_str": "%s_%s_%s.svg",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "title": {
            "format_str": "%s %s %s",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "fig_text": {
            "format_str": "%s %s \nz-source %s \nchisq %.2f ndof %s",
            "arg_keys": ["stock", "model", "redshift_kind", "chisq", "ndof"]
        },
        "width": 10,
        "height": 6,
        "id_mapper": "ZTFIdMapper",
        "disk_save": "/home/jnordin/tmp/ztfsalt",
    }
   
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "SimpleDecentFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2DigestRedshifts",
                                    "config": ampelz_conf,
                                },
                                {
                                    "unit": "T2RunSncosmo",
                                    "config": sncosmo_conf,
                                },
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                        ],
                    }

                }
            }
        }
    ]

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = ITER_MAX,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': "avro",
            'loader': {
                'unit': 'TarAlertLoader',
                'config': {'file_path': ALERT_ARCHIVE}
            }
        }
    },
    directives = directives
)


In [ ]:
n = ac.run()

In [ ]:
print(f"{n} alerts processed for channel {channel}")

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

In [ ]:
table_config =  {
                                        'table_schema' : {
                                            'T2DigestRedshifts': {
                                                'Ampel z' : ['ampel_z'],
                                                'Ampel z group' : ['group_z_nbr'],
                                            },
                                            'T2RunSncosmo' : {
                                                'Peak brightness (abs mag)' : ['fit_metrics','restpeak_model_absmag_B'],
                                                'SALT2 X1' : ['sncosmo_result','paramdict','x1'],
                                                'SALT2 X1 (err)' : ['sncosmo_result','errors','x1'],
                                                'SALT2 Color' : ['sncosmo_result','paramdict','c'],
                                                'SALT2 Color (err)' : ['sncosmo_result','errors','c'],
                                                'Pulls around peak' : ['fit_metrics','absmean_peak_pull'],
                                                'Det. around peak' : ['fit_metrics','nbr_peak_pulls'],
                                            },
                                        },
                                        'transient_table_schema' : {
                                            'T2CatalogMatch': {
                                                'SDSS spec z' : ['SDSS_spec','z'],
                                                "NED z" : ['NEDz','z'],
                                                "NED dist" : ['NEDz','dist2transient'],
                                                "Glade z" : ['GLADEv23','z'],
                                                "LS_Zou Photo-z": ['LSPhotoZZou','photoz'],
                                                "LS_Zou dist": ['LSPhotoZZou','dist2transient'],
                                                "WiseCos Photo-z": ['wiseScosPhotoz','zPhoto_Corr'],
                                                "WiseCos dist": ['wiseScosPhotoz','dist2transient'],
                                            },
                                        },
                                        'include_stock' : True,
                                        'include_channels' : True,
                                        'fmt' : 'csv',
                                        'local_path' : '/home/jnordin/tmp'
                                    }

In [ ]:
t2_filter =  {'all_of':
              [
                {
                    'unit': 'T2DigestRedshifts',
                    'match': {
                        'ampel_z': {"$lt":2.6},
                        'group_z_nbr': {"$lt":7},
                    }
                },
             ]
    }

In [ ]:
t3p = ctx.new_context_unit(
    process_name = "LensPrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3StockSelector",
                            "config": {"channel": channel}
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {"unit": "TransientInfoPrinter"},
                            {
                                "unit": "TransientTablePublisher",
                                "config": table_config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()